In [26]:
import os
import numpy as np
import cv2

<h1> Gaussian and Laplacian Pyramids </h1>
<p> In this notebook we will be implementing a Gaussian and a Laplacian Pyramid. </p>
<p> For testing purposes we will be picking up our image from the "seed_images" folder. If you want to run this experiment for another image, store your test image in that folder and call the method readImage with the name of your image.</p>

In [27]:
def readImage(imageName):
    return cv2.imread("./seed_images/"+imageName)

<p> We also define a method to store our results into as follows: </p>

In [28]:
def createFolder(name):
    try:
        os.mkdir(name)
        print("Created folder with the name \"{}\"".format(name))
        return name
    except:
        print("Folder \"{}\" already exists!".format(name))

In [29]:
def createResultFolders(imageName):
    name = "./results/"+imageName.split(".")[0]+"_results"
    createFolder(name)
    createFolder(name+"/gaussian_pyramid")
    createFolder(name+"/laplacian_pyramid")
    createFolder(name+"/log_pyramid")
    return name

<p> Change the name of imageName to your image name to test for an image of your choice </p>

In [30]:
#imageName = "your_image_name"
imageName="messi.jpg"
image = readImage(imageName)
imageLabel = imageName.split(".")[0]
folderName = createResultFolders(imageName)

Folder "./results/messi_results" already exists!
Folder "./results/messi_results/gaussian_pyramid" already exists!
Folder "./results/messi_results/laplacian_pyramid" already exists!
Created folder with the name "./results/messi_results/log_pyramid"


<p> Before we go ahead and perform any of our operations, we first check the type of image we've read. If it's a coloured image (RGB), we convert it to Greyscale Image, if not, we proceed to the next step. </p>

</br>
<p> First we define our RGB to Greyscale Function </p>
<p> We convert RGB to Greyscale with the following Equation:<p> </br> 
<p>New grayscale image = <i>((0.3*R) + (0.59*G) + (0.11*B))</i></p>
<p> You can read more about it <a href="https://www.tutorialspoint.com/dip/grayscale_to_rgb_conversion.htm"> here </a>. </p>

In [31]:
def rgb2gray(image):
    r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

<p> We perform our check on the basis of the number channels in our input image. If we have 3D array, we've received 3 channels RGB, hence we convert it. If not, we've received an already Grayscale image, and hence return it as it is. </p>

In [32]:
def convert2GrayScaleIfNeeded(image):
    if(len(image.shape)>2):
        print("Converting to Greyscale..")
        return rgb2gray(image)
    else:
        print("Returning already Grayscale image!")
        return image

In [33]:
greyImage = convert2GrayScaleIfNeeded(image)

Converting to Greyscale..


<p> Now we're ready to perform the necessary processing to get our Gaussian and Laplacian Pyramids </p>

<h2> Helper Functions </h2>
<p> Before we jump into building our Gaussian and Laplacian Pyramids we first define a few helper functions that we can call </p>
<p> We start off by defining our Gaussian Blur Function in order to get a blurred Image. But before that we first define the Gaussian Kernel, which we will be using as our filter to blur the Images. The design choice we've made here is to use a 3x3 Gaussian Filter. The choice of kernel is as seen below: </p> <br/>
<img src="kernel.svg" alt="3x3 Gaussan Kernel"> </img>
<br/>
<p> You can read more about it <a href = "https://en.wikipedia.org/wiki/Kernel_(image_processing)"> here</a>.</p>

In [34]:
gaussianKernel = np.array([[1, 2, 1],[ 2, 4, 2], [1, 2, 1]])
gaussianScale = 16.0

<p> The following Kernel is used for the Laplacian Convolving </p>
<img src="lapkernel.jpg" alt="3x3 Laplacian Kernel"> </img>
<p>You can read more about it <a href="">here</a>.</p>

In [35]:
laplacianKernel = np.array([[0, 1, 0],[ 1, -4, 1], [0, 1, 0]])
laplacianScale = 1.0

<p> Now we define our Convolve Method </p>

In [36]:
def convolve(image, imageFilter, scaleValue):
    blurredList = []
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            filterPixels = imageFilter[max(0, 1-i):imageFilter.shape[0]+min(0, image.shape[0]-(i+2)), max(0, 1-j):imageFilter.shape[1]+min(0, (image.shape[1]-(j+2)))]
            startX = ((j+max(0, 1-j))-1)
            startY = ((i+max(0, 1-i))-1)
            imagePixels = image[startY:startY+filterPixels.shape[0], startX:startX+filterPixels.shape[1]]
            flatImage = np.array(filterPixels).flatten()
            flatFilter = np.array(imagePixels).flatten()
            blurredPixel = (np.dot(flatImage, flatFilter))/scaleValue
            blurredList.append(blurredPixel)
    blurredImage = np.array(blurredList).reshape(image.shape)
    return blurredImage

In [37]:
def gaussianBlur(image):
    return convolve(image, gaussianKernel, gaussianScale)

In [38]:
def laplacian(image):
    return convolve(image, laplacianKernel, laplacianScale)

<p> Next, we define a function to scale down the input image that we can make use of to construct the pyramid. The images are scaled down by discarding all the even rows and columns, thus, getting an image 1/4th the size.</p>

In [39]:
def scaleDownImage(image):
    return image[1:image.shape[0]:2, 1:image.shape[1]:2]

<p> We follow this up by defining a function to scale up the image. This is done by injecting rows and columns of zeros after each row and column in the existing image. Then we apply our Gaussian Filter method as defined earlier and Multiply it by 4. </p>

In [40]:
def scaleUp(image):
    return np.insert(np.insert(image, np.arange(1, image.shape[0]+1), 0, axis=0), np.arange(1, 
                image.shape[1]+1), 0, axis=1)

In [41]:
def scaleUpImage(image):
    scaledImage = scaleUp(image)
    scaledUpImage = gaussianBlur(scaledImage)*4
    return scaledUpImage

<p> Lastly we define our "Difference" function which we can use to get the Laplacian. We just return the difference of two images. </p>

In [42]:
def imageDifference(image_1, image_2):
    return np.subtract(image_1[0:image_2.shape[0], :image_2.shape[1]], image_2)

<h2>Constructing the Pyramids </h2>

<p> With the image pre-processed and helper functions defined, we can finally begin constructing our Gaussian and Laplacian Pyramids </p>

<p> We construct our Gaussian And Laplacian Pyramids by doing the following steps: </p>
<ol>
    <li> Input Image </li>
    <li> Blur Image using the Gaussian Blur Function </li>
    <li> Scale the Image down</li>
    <li> Save the Scaled down image into the results/image_name/gaussian_pyramid Folder </li>
    <li> Scale the above image back up </li>
    <li> Calculate the difference between the above Blurred, Scaled-Down, Scaled-up Image and the Input Image for the given level </li>
    <li> Save the difference into the results/image_name/laplacian_pyramid Folder </li>
</ol>
<p> We repeat the above steps for N times (defaulted to 5) </p> 

In [45]:
def constructPyramids(image, imageLabel, folderName, N=5):
    levelImage = image
    gaussianPath = folderName+"/gaussian_pyramid/"+imageLabel+"_gaussian_level_"
    laplacianPath = folderName+"/laplacian_pyramid/"+imageLabel+"_laplacian_level_"
    logPath = folderName+"/log_pyramid/"+imageLabel+"_log_level_"
    for i in range(N):
        blurredLevelImage = gaussianBlur(levelImage)
        scaledDownLevelImage = scaleDownImage(blurredLevelImage)
        cv2.imwrite(gaussianPath+str(i)+".jpg", scaledDownLevelImage)
        scaledUpLevelImage = scaleUpImage(scaledDownLevelImage)
        differenceImage = imageDifference(levelImage, scaledUpLevelImage)
        cv2.imwrite(laplacianPath+str(i)+".jpg", differenceImage)
        logLevelImage = laplacian(blurredLevelImage)
        cv2.imwrite(logPath+str(i)+".jpg", logLevelImage)
        levelImage = scaledDownLevelImage

<p> Finally we can call our "constructPyramids" function that makes use of all the helper functions we've defined earlier. You can add an extra parameter to set the number of levels if you want to increase or decrease it. </p>

In [46]:
constructPyramids(greyImage, imageLabel, folderName)

<p> You can view your results by going to: results/<your_image_name>_results </p>

<h1> The End </h1>